In [16]:
import xlrd
import xlsxwriter
from vaderSentiment import vaderSentiment as vs

In [76]:
class TestCaseStatus:
    """
    Documentation: 
    This class is designed to predict the status of executed testcase whether it is pass/fail 
    and also to generate the new file with predicted status.
    """
    def __init__(self, workbookName, worksheetIndex):
        self.workbookName = workbookName
        self.worksheetIndex = worksheetIndex
    
    def getWorksheet(self):
        workbook = xlrd.open_workbook(self.workbookName)
        worksheet = workbook.sheet_by_index(self.worksheetIndex)
        return worksheet 
    
    def getActualOutput(self, worksheet):
        acutalOutput_ColIndex = list(map(lambda x:x.lower(),worksheet.row_values(0))).index('Actual Output'.lower())
        actualOutputs = worksheet.col_values(acutalOutput_ColIndex )[1:]
        return actualOutputs
    
    def getPredictionResult(self):
        worksheet = self.getWorksheet()
        actualOutputs = self.getActualOutput(worksheet)
        sa = vs.SentimentIntensityAnalyzer()
        result = list(map(lambda x : "Not Executed" if not x else "Failed" if sa.polarity_scores(x)['compound'] <= -0.05 else "Passed" , actualOutputs))
        result.insert(0, "Status- Pass/Fail")
        return result
    
    def drawChart(self, newWorkbook, newWorksheet, result):
        status = [ (i,s, result.count(s)) for i,s in enumerate(["Passed", "Failed", "Not Executed"])]
        lastRow = len(result)+5
        newWorksheet.write_row(lastRow-1, 0, ["Status","Count"], newWorkbook.add_format({'bold': 1}))
        for i,s,count in status:
            newWorksheet.write(lastRow+i, 0, s)
            newWorksheet.write(lastRow+i, 1, count)
        
        chart = newWorkbook.add_chart({'type': 'pie'})
        chart.add_series({
            'name':       'Pie testcase data',
            'categories': ['Sheet1', lastRow, 0, lastRow+2, 0],
            'values':     ['Sheet1', lastRow, 1, lastRow+2, 1],
        })
        
        chart.set_title({'name': 'Test Case Execution Status'})
        chart.set_style(10)
        newWorksheet.insert_chart('C2', chart, {'x_offset': 50, 'y_offset': lastRow*15})   
    
    def generateFile(self, result, fileName):
        worksheet = self.getWorksheet()
        data = [[worksheet.cell_value(r, c)  if c!=worksheet.ncols else result[r] for c in range(worksheet.ncols+1) ] for r in range(worksheet.nrows)]
        newWorkbook = xlsxwriter.Workbook(fileName)
        newWorksheet = newWorkbook.add_worksheet()
        cellFormat = newWorkbook.add_format()
        cellFormat.set_pattern(1)
        cellFormat.set_bg_color('#ff0000')
        
        for i,row in enumerate(data):
            if row[-1] == "Failed":
                newWorksheet.write_row(i, 0, row, cellFormat)
            else:
                newWorksheet.write_row(i, 0, row)
       
        self.drawChart(newWorkbook,newWorksheet,result)
     
        newWorkbook.close()

In [77]:
WORKBOOK_NAME = "testcase.xlsx"
WORKSHEET_INDEX = 0
NEWFILE_NAME = 'testcaseStatus.xlsx'
tcS = TestCaseStatus(WORKBOOK_NAME, WORKSHEET_INDEX)
result = tcS.getPredictionResult()
tcS.generateFile(result, NEWFILE_NAME)

In [78]:
print(tcS.__doc__)


    Documentation: 
    This class is designed to predict the status of executed testcase whether it is pass/fail 
    and also to generate the new file with predicted status.
    
